In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import pprint
import pandas as pd

In [2]:
# To do:
# Need to grab those 'what properties and views do i have access to functions
# Need to build a 'this can't change in a report' function (view, date, segment, etc.)
# Need to build a build_request body function (this is partially happening in the get the data function)
# And related, need to build a request list building
# Augment/build function to optionally return the max/min value thing


def get_service_client(scopes, api_name, api_version, service_key_file=None, credentials=None):
    
    if credentials:
        service_client = build(api_name, api_version, credentials=credentials)
    
    else:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(service_key_file, scopes=scopes)
        service_client = build(api_name, api_version, credentials=credentials)
    
    return service_client

def get_analytics_data(view_id, date_range, reporting_client, request_body):

    for request in request_body['reportRequests']:
        request.update({'viewId': view_id})
        request.update({'dateRanges': [date_range]})

    data_output = reporting_client.reports().batchGet(
        body=request_body
        ).execute()

    return data_output

In [3]:
scopes = ['https://www.googleapis.com/auth/analytics.readonly','https://www.googleapis.com/auth/analytics.edit']
service_credentials_file = 'client_secrets.json'
api_name = 'analyticsreporting'
api_version = 'v4'

reporting_client = get_service_client(scopes, api_name, api_version, service_credentials_file)

In [4]:
view_id = '95100800'
date_range = {'startDate': '28daysAgo', 'endDate': 'today'}
# So it is two in the request body
request_body = {
        'reportRequests': [
            {
              'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:pageviews'}],
              'dimensions': [{'name': 'ga:country'},{'name': 'ga:browser'}]
            }]
        }
#             {
#               'metrics': [{'expression': 'ga:sessions'}],
#               'dimensions': [{'name': 'ga:browser'}]
#             }]
#     }

In [5]:
report_output = get_analytics_data(view_id, date_range, reporting_client, request_body)
report_output

{'reports': [{'columnHeader': {'dimensions': ['ga:country', 'ga:browser'],
    'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
       'type': 'INTEGER'},
      {'name': 'ga:pageviews', 'type': 'INTEGER'}]}},
   'data': {'maximums': [{'values': ['4', '10']}],
    'minimums': [{'values': ['1', '1']}],
    'rowCount': 5,
    'rows': [{'dimensions': ['Brazil', 'Chrome'],
      'metrics': [{'values': ['1', '1']}]},
     {'dimensions': ['Germany', 'Internet Explorer'],
      'metrics': [{'values': ['1', '1']}]},
     {'dimensions': ['Italy', 'Safari'], 'metrics': [{'values': ['1', '1']}]},
     {'dimensions': ['Philippines', 'Chrome'],
      'metrics': [{'values': ['1', '1']}]},
     {'dimensions': ['United States', 'Chrome'],
      'metrics': [{'values': ['4', '10']}]}],
    'totals': [{'values': ['8', '14']}]}}]}

In [6]:
pp = pprint.PrettyPrinter()

pp.pprint(report_output)

{'reports': [{'columnHeader': {'dimensions': ['ga:country', 'ga:browser'],
                               'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions',
                                                                         'type': 'INTEGER'},
                                                                        {'name': 'ga:pageviews',
                                                                         'type': 'INTEGER'}]}},
              'data': {'maximums': [{'values': ['4', '10']}],
                       'minimums': [{'values': ['1', '1']}],
                       'rowCount': 5,
                       'rows': [{'dimensions': ['Brazil', 'Chrome'],
                                 'metrics': [{'values': ['1', '1']}]},
                                {'dimensions': ['Germany', 'Internet Explorer'],
                                 'metrics': [{'values': ['1', '1']}]},
                                {'dimensions': ['Italy', 'Safari'],
                      

In [31]:
dims = report_output.get('reports')[0].get('columnHeader').get('dimensions')
print(dims)
metrics = report_output.get('reports')[0].get('columnHeader').get('metricHeader').get('metricHeaderEntries')
metrics
# metrics[0].get('name')


['ga:country', 'ga:browser']


{'name': 'ga:pageviews', 'type': 'INTEGER'}

In [28]:
metric_list = [metric_header for metric_header in metrics]
metric_list

[{'name': 'ga:sessions', 'type': 'INTEGER'},
 {'name': 'ga:pageviews', 'type': 'INTEGER'}]

In [39]:
metric_headers = []
for metric in metric_list:
    metric_headers.append(metric.get('name'))

metric_headers

['ga:sessions', 'ga:pageviews']

In [43]:
all_heads = dims + metric_headers
all_heads

['ga:country', 'ga:browser', 'ga:sessions', 'ga:pageviews']

In [36]:
def output_to_df(data_output):

    # Yank out the meta data for each report
    for report in data_output.get('reports', []):
        headers = report.get('columnHeader', {})
        dimension_headers = headers.get('dimensions', [])
        metric_details = headers.get('metricHeader', {}).get('metricHeaderEntries', [])
        
        # Run through to grab metric headers
        metric_headers = []
        for metric in metric_details:
            metric_headers.append(metric.get('name'))
        
        column_headers = zip()
        report_headers = [for dim in dimensions_headers]
        # For each report, yank out the actual data values
        for row in report.get('data').get('rows'):
            
#     Add some flavor in here that drops the ga: and cases the first letter
        

SyntaxError: invalid syntax (<ipython-input-36-49a245b966a4>, line 13)

In [ ]:
data_list = []

for row in data_rows:
    print("Row:", row)
    data_row = []
    
    for dimension in row.get('dimensions'):
        print("Dim:", dimension)
        data_row.append(dimension)
        print("Appended row end of Dim:", data_row)
        
    for metric in row.get('metrics')[0].get('values'):
        print("Met:", metric)
        data_row.append(metric)
        print("Append row end of Val", data_row)
    
    data_list.append(data_row)
        
test_data_df = pd.DataFrame(data_list)
test_data_df

In [7]:
report_output.get('reports')[0].get('data').get('rows')

[{'dimensions': ['Brazil', 'Chrome'], 'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Germany', 'Internet Explorer'],
  'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Italy', 'Safari'], 'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Philippines', 'Chrome'],
  'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['United States', 'Chrome'],
  'metrics': [{'values': ['4', '10']}]}]

In [8]:
# Testing to make sure the format I'm making can load into the df alright
test_list = [['Brazil', 'Chrome', '1', '1'],
             ['US', 'IE', '4', '1'],
             ['Germany', 'FF', '8', '6'],
             ['Philippines', 'Chrome', '3', '2']]

test_header = ['Country', 'Browser', 'Sessions', 'Pageviews']

test_df = pd.DataFrame(test_list, columns=test_header)
test_df

,Country,Browser,Sessions,Pageviews
0,Brazil,Chrome,1,1
1,US,IE,4,1
2,Germany,FF,8,6
3,Philippines,Chrome,3,2


In [9]:
data_rows = report_output.get('reports')[0].get('data').get('rows')
pp.pprint(data_rows)

[{'dimensions': ['Brazil', 'Chrome'], 'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Germany', 'Internet Explorer'],
  'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Italy', 'Safari'], 'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['Philippines', 'Chrome'], 'metrics': [{'values': ['1', '1']}]},
 {'dimensions': ['United States', 'Chrome'],
  'metrics': [{'values': ['4', '10']}]}]


In [ ]:
headers = report_output.get('reports')[0].get('columnHeader')
headers
# headers.get('dimensions')
headers.get('metricHeader')

# pp.pprint(report_output.get('reports')[0].get('data'))

In [ ]:
# take the list comprehension approach
dim_headers = [header for header in headers.get('dimensions')]
dim_headers

In [ ]:
def print_response(response):
  
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
        
        for header, dimension in zip(dimensionHeaders, dimensions):
            print(header + ': ' + dimension)
        
        for i, values in enumerate(dateRangeValues):
            print('Date range: ' + str(i))
            
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                print(metricHeader.get('name') + ': ' + value)

In [ ]:
print_response(report_output)

In [ ]:

#Parse the response of API
def parse_response(report):

    """Parses and prints the Analytics Reporting API V4 response"""
    #Initialize results, in list format because two dataframes might return
    result_list = []

    #Initialize empty data container for the two dateranges (if there are two that is)
    data_csv = []
    data_csv2 = []

    #Initialize header rows
    header_row = []

    #Get column headers, metric headers, and dimension headers.
    columnHeader = report.get('columnHeader', {})
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    dimensionHeaders = columnHeader.get('dimensions', [])

    #Combine all of those headers into the header_row, which is in a list format
    for dheader in dimensionHeaders:
        header_row.append(dheader)
    for mheader in metricHeaders:
        header_row.append(mheader['name'])

    #Get data from each of the rows, and append them into a list
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
        row_temp = []
        dimensions = row.get('dimensions', [])
        metrics = row.get('metrics', [])
        for d in dimensions:
            row_temp.append(d)
        for m in metrics[0]['values']:
            row_temp.append(m)
            data_csv.append(row_temp)

        #In case of a second date range, do the same thing for the second request
        if len(metrics) == 2:
            row_temp2 = []
            for d in dimensions:
                row_temp2.append(d)
            for m in metrics[1]['values']:
                row_temp2.append(m)
            data_csv2.append(row_temp2)
    
#     Looking at their output
    print(data_csv)
    
    #Putting those list formats into pandas dataframe, and append them into the final result
    result_df = pd.DataFrame(data_csv, columns=header_row)
    result_list.append(result_df)
    if data_csv2 != []:
        result_list.append(pandas.DataFrame(data_csv2, columns=header_row))

    return result_list

In [ ]:
result = parse_response(report_output)

In [ ]:
result